In [1]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/train_sample.zip

--2021-04-07 13:51:48--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/train_sample.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 118.31.232.194
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|118.31.232.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 540689175 (516M) [application/zip]
Saving to: ‘train_sample.zip’

100%[======================================>] 540,689,175 10.6MB/s   in 53s    

2021-04-07 13:52:42 (9.67 MB/s) - ‘train_sample.zip’ saved [540689175/540689175]



In [2]:
!unzip -qq train_sample.zip
!\rm train_sample.zip

In [3]:
!wget http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/test_a.zip

--2021-04-07 13:54:15--  http://tianchi-competition.oss-cn-hangzhou.aliyuncs.com/531887/test_a.zip
Resolving tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)... 118.31.232.194
Connecting to tianchi-competition.oss-cn-hangzhou.aliyuncs.com (tianchi-competition.oss-cn-hangzhou.aliyuncs.com)|118.31.232.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1092637852 (1.0G) [application/zip]
Saving to: ‘test_a.zip’

100%[====================================>] 1,092,637,852 9.31MB/s   in 1m 50s 

2021-04-07 13:56:05 (9.45 MB/s) - ‘test_a.zip’ saved [1092637852/1092637852]



In [4]:
!unzip -qq test_a.zip
!\rm test_a.zip

## 环境要求

- TensorFlow的版本：2.0 + 
- keras
- sklearn
- librosa

In [5]:
# 基本库

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

## 加载深度学习框架

In [6]:
# 搭建分类模型所需要的库

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## 加载音频处理库

In [11]:
!pip install librosa --user

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
  Using cached librosa-0.8.0-py3-none-any.whl
  Using cached https://mirrors.aliyun.com/pypi/packages/eb/f2/3cbbbf3b96fb9fa91582c438b574cff3f45b29c772f94c400e2c99ef5db9/SoundFile-0.10.3.post1-py2.py3-none-any.whl (21 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/40/b9/9876662636ba451d4406543047c0b45ca5b4e830f931308c8274dad1db43/pooch-1.3.0-py3-none-any.whl (51 kB)
  Using cached audioread-2.1.9-py3-none-any.whl
  Using cached resampy-0.2.2-py3-none-any.whl
  Using cached https://mirrors.aliyun.com/pypi/packages/3b/00/2344469e2084fb287c2e0b57b72910309874c3245463acd6cf5e3db69324/appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)


In [13]:
# 其他库

import os
import librosa
import librosa.display
import glob 

## 特征提取以及数据集的建立

In [14]:
feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2,'candied_fruits':3, 'carrots': 4, 'chips':5,
                  'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream':11,
                  'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon':17,
                  'soup': 18, 'wings': 19}
label_dict_inv = {v:k for k,v in label_dict.items()}

In [49]:
from tqdm import tqdm
def extract_features(parent_dir, sub_dirs, max_file=10, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]): # 遍历数据集的所有文件
            
           # segment_log_specgrams, segment_labels = [], []
            #sound_clip,sr = librosa.load(fn)
            #print(fn)
            label_name = fn.split('/')[-2]
            label.extend([label_dict[label_name]])
            X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
            mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
            feature.extend([mels])
            
    return [feature, label]

In [74]:
# 自己更改目录
parent_dir = './train_sample/'
save_dir = "./"
folds = sub_dirs = np.array(['aloe','burger','cabbage','candied_fruits',
                             'carrots','chips','chocolate','drinks','fries',
                            'grapes','gummies','ice-cream','jelly','noodles','pickles',
                            'pizza','ribs','salmon','soup','wings'])

# 获取特征feature以及类别的label
temp = extract_features(parent_dir,sub_dirs,max_file=100)

100%|██████████| 35/35 [00:08<00:00,  4.08it/s]


In [75]:
temp = np.array(temp)
data = temp.transpose()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [76]:
# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1])
print('X的特征尺寸是：',X.shape)
print('Y的特征尺寸是：',Y.shape)

X的特征尺寸是： (1000, 128)
Y的特征尺寸是： (1000,)


In [77]:
# 在Keras库中：to_categorical就是将类别向量转换为二进制（只有0和1）的矩阵类型表示
Y = to_categorical(Y)

In [78]:
'''最终数据'''
print(X.shape)
print(Y.shape)

(1000, 128)
(1000, 20)


In [79]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1, stratify=Y)
print('训练集的大小',len(X_train))
print('测试集的大小',len(X_test))

训练集的大小 750
测试集的大小 250


In [80]:
X_train = X_train.reshape(-1, 16, 8, 1)
X_test = X_test.reshape(-1, 16, 8, 1)

## 建立模型

### 搭建CNN网络

In [81]:
model = Sequential()

# 输入的大小
input_dim = (16, 8, 1)

model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))# 卷积层
model.add(MaxPool2D(pool_size=(2, 2)))# 最大池化
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh")) #卷积层
model.add(MaxPool2D(pool_size=(2, 2))) # 最大池化层
model.add(Dropout(0.1))
model.add(Flatten()) # 展开
model.add(Dense(1024, activation = "tanh"))
model.add(Dense(20, activation = "softmax")) # 输出层：20个units输出20个类的概率

# 编译模型，设置损失函数，优化方法以及评价标准
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [82]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 2, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)             

In [84]:
# 训练模型
model.fit(X_train, Y_train, epochs = 20, batch_size = 15, validation_data = (X_test, Y_test))

Epoch 1/20
50/50 [==============================] - 3s 49ms/step - loss: 1.5272 - accuracy: 0.5973 - val_loss: 3.6820 - val_accuracy: 0.2960
Epoch 2/20
50/50 [==============================] - 2s 47ms/step - loss: 1.4931 - accuracy: 0.6013 - val_loss: 3.5160 - val_accuracy: 0.3400
Epoch 3/20
50/50 [==============================] - 2s 43ms/step - loss: 0.9785 - accuracy: 0.7027 - val_loss: 3.2466 - val_accuracy: 0.3400
Epoch 4/20
50/50 [==============================] - 2s 42ms/step - loss: 0.6756 - accuracy: 0.7880 - val_loss: 3.0725 - val_accuracy: 0.4080
Epoch 5/20
50/50 [==============================] - 2s 43ms/step - loss: 0.5808 - accuracy: 0.8333 - val_loss: 3.3328 - val_accuracy: 0.3880
Epoch 6/20
50/50 [==============================] - 2s 40ms/step - loss: 0.5391 - accuracy: 0.8507 - val_loss: 3.4656 - val_accuracy: 0.3640
Epoch 7/20
50/50 [==============================] - 2s 41ms/step - loss: 0.4948 - accuracy: 0.8520 - val_loss: 3.6385 - val_accuracy: 0.3680
Epoch 8/20
50

### 预测测试集

In [85]:
def extract_features(test_dir, file_ext="*.wav"):
    feature = []
    for fn in tqdm(glob.glob(os.path.join(test_dir, file_ext))[:]): # 遍历数据集的所有文件
        X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
        mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
        feature.extend([mels])
    return feature

In [87]:
X_test = extract_features('./test_a/')

100%|██████████| 2000/2000 [09:25<00:00,  2.90it/s]


In [88]:
X_test = np.vstack(X_test)
predictions = model.predict(X_test.reshape(-1, 16, 8, 1))

In [92]:
preds = np.argmax(predictions, axis = 1)
preds = [label_dict_inv[x] for x in preds]

path = glob.glob('./test_a/*.wav')
result = pd.DataFrame({'name':path, 'label': preds})

result['name'] = result['name'].apply(lambda x: x.split('/')[-1])
result.to_csv('submit.csv',index=None)

In [90]:
!ls ./test_a/*.wav | wc -l

2000


In [91]:
!wc -l submit.csv

2001 submit.csv
